This is my hands-on guide to using regular expressions in python. The idea is to do as many different examples as possible and develop a practical understanding of how they work. This will largely be a complementary guide and not a substitute for other sources such as [this python documentation](https://docs.python.org/3/library/re.html#module-re)

In [1]:
import re

## Example 1

Let us consider the image file with the following path: ```/home/prarit/.fastai/data/oxford-iiit-pet/images/Egyptian_Mau_192.jpg```. This is supposed to a picture of an Egyptian Mau cat. There images of different breeds of dogs and cat in the same folder. The name of their corresponding breed is in the name of the file itself. The naming has the convention: ```breed_name_number.jpg```. In this exercise we will learn how to write a regular expression to extract the breed name from the file name. 

#### Sol:

The first thing which we notice is that the end of the file name has ```.jpg``` which is preceeded by a number. Let us first learn how to match this piece in the filename. To do so, recall that we can match the end-of-string by using ```$``` in our regex. Therefore to find ```.jpg``` at the end of the string we simply use teh regex ```r'.jpg$' ```. Here the prefix ```r``` stands for a raw string. 

In [2]:
strng = '/home/prarit/.fastai/data/oxford-iiit-pet/images/Egyptian_Mau_192.jpg'
pat = re.compile(r'.jpg')
gp = pat.search(strng)
gp

<re.Match object; span=(65, 69), match='.jpg'>

Great! We found a match. The value of span i.e. (65,69) in the above output tells us that the match was found begining between the 65th and 69th position in our string. This is indeed true as can be seen by outputting the 65th to 69th characters of the string.  

In [3]:
strng[65:69]

'.jpg'

Let us now extract the number that preceeds ```.jpg``` in the filename. To do this recall, that the regular expression ``` r'\d' ``` matches with any decimal digit i.e. ```[0-9]``` in ASCII and also other digit characters in Unicode. Therefore let us try to use ```r'\d.jpg$' ``` as our regex. 

In [4]:
pat = re.compile(r'\d.jpg$')
gp = pat.search(strng)
gp

<re.Match object; span=(64, 69), match='2.jpg'>

This matches with ```2.jpg``` but we wanted ```192.jpg```. More generally the number preceding ```.jpg``` can be an arbitrary multidigit number. Thus we want to consider a regex which will match with one or more digits preceeding ```.jpg```. To do this we recall that a repetition of one or more can specified by a ```+``` character in the regex. Thus we should try a regex ```r'\d+.jpg$'``` .

In [5]:
pat = re.compile(r'\d+.jpg$')
gp = pat.search(strng)
gp

<re.Match object; span=(62, 69), match='192.jpg'>

Great! This matches exactly with our expected output. It basically started matching from the end of the string and stopped once a match was not found anymore. In this, case, moving from right to left, it first looked for ```.jpg```, then it started looking for decimal digits and stopped as soon as it encountered a non-digit character, in this case an underscore i.e. ```_```. 

Next, we want to extract the breed_name preceeding the number. The breed name contains alphabets i.e. ```a-z``` and ```A-Z```. It can also possibly contain underscores. This corresponds to specifying the set ```[a-zA-Z_]``` in our regex. Also, we will again use ```+``` to ensure it matches with one or more sequential occurrances of the characters specified in the above set. Thus, we will try the regex ```r'[a-zA-Z_]+\d+.jpg'```

In [6]:
pat = re.compile(r'[a-zA-Z_]+\d+.jpg')
gp = pat.search(strng)
gp

<re.Match object; span=(49, 69), match='Egyptian_Mau_192.jpg'>

Nice! It again gives us the output we expect. Note that if this is all that we wanted, then we could have constructed a much simpler regex by using ```\w``` along with ```+``` to ensure looking for one or more occurances of all alpha-numeric characters along with underscores. 

In [7]:
pat = re.compile(r'\w+.jpg')
gp = pat.search(strng)
gp

<re.Match object; span=(49, 69), match='Egyptian_Mau_192.jpg'>

However, here our objective is slightly different. Recall that we wish to extract the breed_name from the file name while the rest. This can be done by using ```(...)``` to enclose the piece of regex that is expected to form our breed name. This effective forms a group out of them which can be easily extracted after the match has been performed. We thus try the regex ```r'([a-zA-Z_]+)\d+.jpg'```

In [8]:
pat = re.compile(r'([a-zA-Z_]+)\d+.jpg')
gp = pat.search(strng)

In [9]:
print('The full string matched with the regex is: {}'.format(gp[0]))

The full string matched with the regex is: Egyptian_Mau_192.jpg


In [10]:
print('The string matching with the grp enclosed within (...)'\
      ' in the regex is: {}'.format(gp[1]))

The string matching with the grp enclosed within (...) in the regex is: Egyptian_Mau_


In case you don't want the underscore at the end of the breed_name, it can be separately included in the regex (between ```(...)``` and ```\d```) which then becomes ```r'([a-zA-Z_]+)_\d+.jpg'```

In [11]:
pat = re.compile(r'([a-zA-Z_]+)_\d+.jpg')
gp = pat.search(strng)
print('The string matching with the grp enclosed within (...)'\
      ' in the regex is: {}'.format(gp[1]))

The string matching with the grp enclosed within (...) in the regex is: Egyptian_Mau


More generally, we can form multiple different groups in the regex which can then be accessed using their corresponding index. ```0``` being the full string which matches with the regex, ```1``` being the index of the 1st group in the regex, then ```2``` for the second group and so on ...

For example in the following regex, we can seperately extract the breed_name and image number

In [12]:
pat = re.compile(r'([a-zA-Z_]+)_(\d+).jpg')
gp = pat.search(strng)
print('The breed name is the first group in the regex(...): {}'.format(gp[1]))
print('The breed name is the second group in the regex(...): {}'.format(gp[2]))

The breed name is the first group in the regex(...): Egyptian_Mau
The breed name is the second group in the regex(...): 192


#### A slightly different and more general regex:

Note that this execise was inspired from a similar piece of code in lesson 1 of fast.ai's machine learning [course (v3)](https://course.fast.ai/). There they used a slightly different regex. There, the logic they used is that upond reading from right to left, the breed name corresponds to the string that lies between ```_number.jpg``` and the slash ```/``` in the path to the file. Thus instead of using ```[a-zA-Z_]+``` , we use ```[^/]+``` to look for the breed name. The set ```[^/]``` matches with anything except a ```/```, the ```+``` makes sure that it matches with one or more characters that are not ```/```. Thus the regex we use is ```r'([^/]+)_(\d+).jpg'```

In [13]:
pat = re.compile(r'([^/]+)_\d+.jpg')
gp = pat.search(strng)
print('The full string matching with the regex is: {}'.format(gp[0]))
print('The breed name is the first group in the regex(...): {}'.format(gp[1]))

The full string matching with the regex is: Egyptian_Mau_192.jpg
The breed name is the first group in the regex(...): Egyptian_Mau
